In [ ]:
##########################################################
# 1. IMPORT ALL PACKAGES
##########################################################
import pandas as pd
import numpy as np
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import math
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt #for plotting
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import multilabel_confusion_matrix
# importing mean() 
from statistics import mean 
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

In [ ]:
data = pd.read_csv("../input/letterrecognition-using-svm/letter-recognition.csv",header=0)

labelencoder = LabelEncoder()
data['letter'] = labelencoder.fit_transform(data['letter']) #ditambah label encoder karena data berisi huruf sehingga akan menghasilkan nan(not a number). Maka, kita harus mengubah huruf" itu menjadi nomor

data.head(10)

In [ ]:
# Check all classes/labels in trainiing data
all_label = set(data['letter'].tolist())
print("All labels: {0}".format(all_label))

# Data distribution for each class
dst_train = Counter(data['letter'])
print(dst_train)

# Plot distribution
sns.countplot(data['letter'])

In [ ]:
##########################################################
# 3. SHARE TO TEST AND TRAIN DATA
##########################################################
# We share train data because this "sample_submission.csv" only has one class/label
x = data.iloc[:, 1:]
y = data['letter'].tolist()

# # Select 10000 rows data as a testing dataset
x_test = x.iloc[0:10000, :].values.astype('float32') # all pixel values 
y_test = y[0:10000] # Select label for testing data
x_train = x.iloc[10000:, :].values.astype('float32') # all pixel values 
y_train = y[10000:]

# View one row data
#df_train.iloc[0:1, 1:].values
data.iloc[0:1, 1:].values

In [ ]:
##########################################################
# 6. TRAIN RANDOM FOREST ALGORITHM
##########################################################
# Create a RandomForestClassifier object with the parameters over the data
# n_estimators (default=100) = the number of trees in the forest.
# max_depth (default=None) = the maximum depth of the tree.
model_clf = SVC(C=100, kernel='poly',gamma=0.1)

# Train the Random Forest algorithm
model_clf.fit(x_train, y_train)

In [ ]:
##########################################################
# 7. APPLY THE TRAINED LEARNER TO TEST NEW DATA
##########################################################
# Apply the trained perceptron to make prediction of test data
y_pred = model_clf.predict(x_test)

In [ ]:
# 7. MULTI-CLASS CONFUSION MATRIX FOR EACH CLASS
##########################################################
# Actual and predicted classes
lst_actual_class = y_test
lst_predicted_class = y_pred

lst_classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25] # Must in order

# Compute multi-class confusion matrix
arr_out_matrix = multilabel_confusion_matrix(lst_actual_class, lst_predicted_class, labels=lst_classes)

# Temp store results
store_sens = [];
store_spec = [];
store_acc = [];
store_bal_acc = [];
store_prec = [];
store_fscore = [];
store_mcc = [];

# Loop for each taget label
for no_class in range(len(lst_classes)):
    arr_data = arr_out_matrix[no_class];
    print("Predicted Performance of Digit Label/Class: {0}".format(no_class));
    tp = arr_data[1][1]
    fp = arr_data[0][1]
    tn = arr_data[0][0]
    fn = arr_data[1][0]

    sensitivity = round(tp/(tp+fn), 3);
    specificity = round(tn/(tn+fp), 3);
    accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
    balanced_accuracy = round((sensitivity+specificity)/2, 3);
    precision = round(tp/(tp + fp), 3)
    fscore = round((2 * ((precision * sensitivity) / (precision + sensitivity))), 3)
    mcc = round((((tp * tn)-(fp * fn))/ math.sqrt((tp + fp)*(tp + fn)*(tn + fp)*(tn + fn))),3)

    store_sens.append(sensitivity);
    store_spec.append(specificity);
    store_acc.append(accuracy);
    store_bal_acc.append(balanced_accuracy);
    store_prec.append(precision);
    store_fscore.append(fscore);
    store_mcc.append(mcc);
    
    print("TP={0}, FP={1}, TN={2}, FN={3}".format(tp, fp, tn, fn));
    print("Sensitivity: {0}".format(sensitivity));
    print("Specificity: {0}".format(specificity));
    print("Accuracy: {0}".format(accuracy));
    print("Balanced Accuracy: {0}".format(balanced_accuracy));
    print("Precision: {0}".format(precision));
    print("F1-Score: {0}".format(fscore));
    print("MCC: {0}\n".format(mcc));

In [ ]:
##########################################################
# 8. OVERALL - FINAL PREDICTION PERFORMANCE
##########################################################
print("Overall Performance Prediction:");
print("Sensitivity: {0}%".format(round(mean(store_sens)*100, 4)));
print("Specificity: {0}%".format(round(mean(store_spec)*100, 4)));
print("Accuracy: {0}%".format(round(mean(store_acc)*100, 4)));
print("Balanced Accuracy: {0}%".format(round(mean(store_bal_acc)*100, 4)));
print("Precision: {0}%".format(round(mean(store_prec)*100, 4)));
print("F1-Score: {0}%".format(round(mean(store_fscore)*100, 4)))
print("MCC: {0}\n".format(round(mean(store_mcc), 4)))